In [1]:
import os
import sys
#nb_dir = os.path.split(os.getcwd())[0]
#if nb_dir not in sys.path:
#    sys.path.append(nb_dir)
    
autolib_dir = '/home/kai/data/shiyi/AlphaBoosting/automl'
if autolib_dir not in sys.path:
    sys.path.append(autolib_dir)


In [2]:
sys.path

['',
 '/home/kai/anaconda3/lib/python36.zip',
 '/home/kai/anaconda3/lib/python3.6',
 '/home/kai/anaconda3/lib/python3.6/lib-dynload',
 '/home/kai/anaconda3/lib/python3.6/site-packages',
 '/home/kai/anaconda3/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/duckling-1.8.0-py3.6.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/JPype1-0.6.3-py3.6-linux-x86_64.egg',
 '/home/kai/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/kai/.ipython',
 '/home/kai/data/shiyi/AlphaBoosting/automl']

In [3]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
from automl_libs import feature_engineering as fe
from automl_libs import encoding, kernels 

In [6]:
# params
#param = {'trainLen': self.train_len, 'splitCol': 'a', 'col': self.label, 'coefficient': 10, 'n': 2, 'fillna': 22}
# feature_engineering_dict.append({'params': params, 'function': 'count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'unique_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'cumulative_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'reverse_cumulative_count', 'feature_cols': ['a','b']})
# feature_engineering_dict.append({'params': params, 'function': 'variance', 'feature_cols': ['a','n']})
# feature_engineering_dict.append({'params': params, 'function': 'count_std_over_mean', 'feature_cols': ['a','b']})


# feature_engineering_dict.append({'params': params, 'function': 'time_to_n_next', 'feature_cols': ['a','t']})
# feature_engineering_dict.append({'params': params, 'function': 'count_in_previous_n_time_unit', 'feature_cols': ['a','t']})
# feature_engineering_dict.append({'params': params, 'function': 'count_in_next_n_time_unit', 'feature_cols': ['a','t']})

# feature_engineering_dict.append({'params': params, 'function': 'woe', 'feature_cols': ['a']})
# feature_engineering_dict.append({'params': params, 'function': 'chi_square', 'feature_cols': ['a']})
# feature_engineering_dict.append({'params': params, 'function': 'mean', 'feature_cols': ['a']})

In [6]:
categorical_features = ['user', 'movie', 'age', 'gender', 'occupation', 'zip', 'released']

In [7]:
import itertools
#params = {'split_col': 't', 'coefficient': 10, 'n': 2, 'fillna': 22}
def get_features_to_gen(function_list):
    features_to_gen = []
    for function in function_list:
        for i in range(2, len(categorical_features)+1):
            for combine in itertools.combinations(categorical_features, i):
                if function.__name__ == 'count_std_over_mean':
                    features_to_gen.append({'params': {'coefficient': 10}, 'function': function, 'feature_cols': list(combine)})
                else:
                    features_to_gen.append({'params': {}, 'function': function, 'feature_cols': list(combine)})
                
    return features_to_gen

features_to_gen = get_features_to_gen([fe.count])#, fe.unique_count, fe.cumulative_count,
                                     #fe.reverse_cumulative_count, fe.variance, fe.count_std_over_mean])
# features_to_gen = get_features_to_gen([])#fe.count])

features_to_gen

[{'feature_cols': ['user', 'movie'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'age'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'gender'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'occupation'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'zip'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['user', 'released'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['movie', 'age'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['movie', 'gender'],
  'function': <function automl_libs.feature_engineering.count>,
  'params': {}},
 {'feature_cols': ['movie', 'occupation'],
  'func

In [8]:
len(features_to_gen)

120

In [9]:
project_path = '/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project3/' 

In [10]:
from automl_app import logger_config

logger_config.config(project_path+'log_project3.log')

In [11]:
def lgb_gs_params_gen():
    import time
    import numpy as np
    seed = int(time.time()* 1000000) % 45234634
    np.random.seed(seed)
    params = {    
        'objective': 'regression',#'binary',
        'boosting': 'gbdt',
        'num_rounds': 20,
        'learning_rate': np.random.choice([0.1,0.03]),#0.001]),
        'num_leaves': np.random.choice([15,31,61,127]),
        'num_threads': 8, # best speed: set to number of real cpu cores, which is vCPU/2
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': np.random.randint(20,50),  #minimal number of data in one leaf. 
        'feature_fraction': np.random.randint(4,11)/10,
        'feature_fraction_seed': seed,
        'early_stopping_round':70,
        'bagging_freq': 1, #0 means disable bagging. k: perform bagging at every k iteration
        'bagging_fraction': np.random.randint(4,11)/10, #Randomly select part of data 
        'bagging_seed': seed,
        'scale_pos_weight': 2,
        'metric' : 'rmse' 
    }
    return params, seed

In [12]:
lgb_gs_params_gen()

({'bagging_fraction': 1.0,
  'bagging_freq': 1,
  'bagging_seed': 14500784,
  'boosting': 'gbdt',
  'early_stopping_round': 70,
  'feature_fraction': 0.7,
  'feature_fraction_seed': 14500784,
  'learning_rate': 0.10000000000000001,
  'max_depth': -1,
  'metric': 'rmse',
  'min_data_in_leaf': 46,
  'num_leaves': 61,
  'num_rounds': 20,
  'num_threads': 8,
  'objective': 'regression',
  'scale_pos_weight': 2},
 14500784)

In [15]:
from automl_app.app import AlphaBoosting

automl_config_file = project_path + 'automl_config.json'
run_record_file_name = project_path + 'run_record.json' # don't created this file
AlphaBoosting(automl_config_file, features_to_gen, lgb_gs_params_gen)

2018-06-18 11:37:16 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #57  | Run record file [./output/run_record.json] found. Continue from the previous run...
2018-06-18 11:37:16 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #88  | save run record
2018-06-18 11:37:16 EST | INFO     | automl_app.app.AlphaBoosting                 | _save_run_record     | #152 | val index is saved at ./log/val_index.json
2018-06-18 11:37:16 EST | INFO     | automl_app.app.AlphaBoosting                 | _save_run_record     | #154 | run record is saved at ./output/run_record.json
2018-06-18 11:37:16 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #92  | generate todo list
2018-06-18 11:37:16 EST | INFO     | automl_app.app.AlphaBoosting                 | __init__             | #108 | feature engineering
2018-06-18 11:37:16 EST | INFO     | automl_app.app.AlphaBoosting                 | __

[1]	cv_agg's rmse: 1.11874 + 9.49562e-05
[2]	cv_agg's rmse: 1.11219 + 0.000204
[3]	cv_agg's rmse: 1.10596 + 0.000284266
[4]	cv_agg's rmse: 1.10007 + 0.000375003
[5]	cv_agg's rmse: 1.09448 + 0.000500362
[6]	cv_agg's rmse: 1.08914 + 0.000577821
[7]	cv_agg's rmse: 1.08405 + 0.000636457
[8]	cv_agg's rmse: 1.07925 + 0.000689649
[9]	cv_agg's rmse: 1.07464 + 0.000745515
[10]	cv_agg's rmse: 1.0703 + 0.000853929
[11]	cv_agg's rmse: 1.06615 + 0.000912623
[12]	cv_agg's rmse: 1.06207 + 0.000952991
[13]	cv_agg's rmse: 1.05823 + 0.00104154
[14]	cv_agg's rmse: 1.05452 + 0.00114908
[15]	cv_agg's rmse: 1.05097 + 0.00122941
[16]	cv_agg's rmse: 1.04755 + 0.0013926


2018-06-18 11:37:18 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #58  | [do_preds] is True, generating predictions ...
2018-06-18 11:37:18 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #60  | Retrain model using cv params and all data...


[17]	cv_agg's rmse: 1.04423 + 0.00147283
[18]	cv_agg's rmse: 1.04107 + 0.00145317
[19]	cv_agg's rmse: 1.03799 + 0.00150125
[20]	cv_agg's rmse: 1.03505 + 0.00160161


2018-06-18 11:37:19 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #64  | Predicting...
2018-06-18 11:37:19 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #69  | Predictions(1529336237) saved in ./output/gs_saved_preds/.
2018-06-18 11:37:19 EST | DEBUG    | automl_libs.grid_search                      | lgb_grid_search      | #90  | ./output/project2_grid_search.csv updated


[4]	training's rmse: 1.09872
[8]	training's rmse: 1.07652
[12]	training's rmse: 1.05786
[16]	training's rmse: 1.04193
[20]	training's rmse: 1.0284
